In [1]:
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np
from PIL import Image, ImageOps
import os

In [2]:
dog_model = keras.models.load_model('dogbreedrecognition.h5')

Metal device set to: Apple M1 Pro


2023-12-18 18:06:41.519508: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-18 18:06:41.519626: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
dog_categories = []

for root, dirs, files in os.walk("/Users/mm22/Downloads/Dog_images"):
    for dir_name in dirs:
        dog_categories.append(dir_name)
dog_categories

['Australian_terrier',
 'toy_poodle',
 'Great_Pyrenees',
 'Maltese_dog',
 'Norwich_terrier',
 'whippet',
 'Boston_bull',
 'Irish_setter',
 'Rottweiler',
 'kelpie',
 'schipperke',
 'Leonberg',
 'Welsh_springer_spaniel',
 'Pomeranian',
 'Pekinese',
 'Irish_wolfhound',
 'Blenheim_spaniel',
 'basenji',
 'African_hunting_dog',
 'komondor',
 'Yorkshire_terrier',
 'basset',
 'Japanese_spaniel',
 'standard_schnauzer',
 'dhole',
 'miniature_pinscher',
 'Lhasa',
 'Walker_hound',
 'Kerry_blue_terrier',
 'standard_poodle',
 'Saint_Bernard',
 'Chihuahua',
 'Afghan_hound',
 'Newfoundland',
 'black-and-tan_coonhound',
 'pug',
 'Scottish_deerhound',
 'cairn',
 'malamute',
 'beagle',
 'vizsla',
 'collie',
 'Italian_greyhound',
 'West_Highland_white_terrier',
 'Brittany_spaniel',
 'English_springer',
 'affenpinscher',
 'Doberman',
 'silky_terrier',
 'Pembroke',
 'Weimaraner',
 'papillon',
 'Norwegian_elkhound',
 'Sussex_spaniel',
 'soft-coated_wheaten_terrier',
 'Shih-Tzu',
 'Ibizan_hound',
 'cocker_spa

In [4]:
# function to prepare the image
def prepare_specificmodel(filepath):
    IMG_SIZE = 299
    img_array = cv2.imread(filepath)[...,::-1]
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

# model recognition function
def identify_dog(image_path):
    prediction = dog_model.predict([prepare_specificmodel(image_path)])
    #print(categories[int(prediction)])
    prediction_list = list(prediction[0])
    max_index = prediction_list.index(max(prediction_list))
    print("Breed: ", dog_categories[max_index], max(prediction_list))

In [5]:
#testing the model recognition. Input the path to an image as demonstrated below
path = '/Users/mm22/Downloads/junk/testing/Pembroke/n02113023_1151.jpg'
identify_dog(path)

2023-12-18 18:07:05.562077: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-12-18 18:07:05.981173: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 5s 5s/step
Breed:  Pembroke 0.84357554
